In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import tensorflow as tf
import keras
from keras.layers.pooling import MaxPooling2D

from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
import pandas as pd
import csv
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.initializers import RandomNormal
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2                 
import os          
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix        
from random import shuffle
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob as gb
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
!pip install pydicom
from tensorflow.keras.applications import EfficientNetB7
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from functools import partial
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Conv2D,BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras import callbacks
import random
import pprint
!pip install pypng
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from keras.layers import Concatenate
from tensorflow.keras.optimizers import SGD

In [ ]:
import numpy as np
x_train = np.load('/content/drive/MyDrive/ĐCLVTN/image/up1_np/224/X.npy')
y_train = np.load('/content/drive/MyDrive/ĐCLVTN/image/up1_np/224/y.npy')

print(y_train.shape)
print(x_train.shape)
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25, 
    rescale=1./4095,)
datagen.fit(x_train)

(1400, 4)
(1400, 224, 224, 3)


In [ ]:
import numpy as np
x_test = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/cut/test/512/X.npy')
y_test = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/cut/test/512/y.npy')
x_test = x_test/4095
print(x_test.shape)
print(y_test.shape)

(835, 512, 512, 1)
(835, 4)


proposed_vgg16

In [ ]:
cb_proposed = tf.keras.callbacks.ModelCheckpoint(
    '/content/VGG16_224_proposed.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
#propesed
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=128,kernel_size=(3,3),padding="same", activation="tanh"))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="tanh"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="tanh"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="tanh"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="tanh"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="tanh"))
#model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
#model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="tanh"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="tanh"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="tanh"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="tanh"))
#model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=512,activation="tanh"))
model.add(Dense(units=256,activation="tanh"))
model.add(Dense(units=4, activation="softmax"))
model.summary()
opt = SGD(momentum = 0.9)
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['binary_accuracy'])

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=32,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb_proposed, 
         epochs=50)
print("Total time: ", time.time() - start, "seconds")

In [ ]:
acc = model1.history['binary_accuracy']                                                                                                         
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

standard_vgg16

In [ ]:
cb1 = tf.keras.callbacks.ModelCheckpoint(
    '/content/VGG16.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
baseModel = VGG16(input_shape=(224, 224,1), weights = None, include_top=False)
for layer in baseModel.layers:
  layer.trainable = True

headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(512, activation="relu")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(256, activation="relu")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(4, activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()
opt = SGD(momentum = 0.9)
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['binary_accuracy'])

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=32,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb1, 
         epochs=50)
print("Total time: ", time.time() - start, "seconds")

In [ ]:
acc = model1.history['binary_accuracy']                                                                                                         
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
loadedModel = keras.models.load_model('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/cut/save_model/VGG16/VGG16_cut_512.h5')
#loadedModel.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['binary_accuracy'])
loadedModel.evaluate(x_test,y_test,batch_size = 10)
y_pre = loadedModel.predict(x_test)
y_pre = np.round(y_pre)
y_pre
y_true = y_test

labels =['1', '2', '3','4']
print(classification_report(y_true, y_pre, target_names=labels))